## Train models
*reference: cell-health/3.train/1.train-models*, in fact this section is directly from that notebook 

In [1]:
import os
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning

from scripts.ml_utils import (
    load_train_test,
    CellHealthPredict,
)  # moved this into my folder and it works!

/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecat

In [2]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [3]:
np.random.seed(123)

### prep data (load in future)

In [4]:
method = "median"
# method = 'weighted'

train_folder = "data/train_test/{}_agg/".format(method)
consensus = "modz"
model_folder = "models"

os.makedirs(f"{model_folder}", exist_ok=True)
os.makedirs(f"{model_folder}/{method}_agg", exist_ok=True)

In [5]:
x_train_df, x_test_df, y_train_df, y_test_df = load_train_test(
    data_dir=train_folder, drop_metadata=True, consensus=consensus
)
x_meta_train_df, x_meta_test_df, y_meta_train_df, y_meta_test_df = load_train_test(
    data_dir=train_folder, output_metadata_only=True, consensus=consensus
)

In [6]:
cell_lines = list(set(x_meta_train_df.Metadata_cell_line))
cell_lines

['ES2', 'HCC44', 'A549']

In [7]:
print(x_train_df.shape)
x_train_df.head(3)

(303, 943)


,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_Orientation,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_3,...,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_SumVariance_AGP_20_0,Nuclei_Texture_SumVariance_AGP_5_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_DNA_20_0,Nuclei_Texture_SumVariance_DNA_5_0,Nuclei_Texture_Variance_AGP_5_0,Nuclei_Texture_Variance_DNA_10_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_DNA_5_0
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_153,-0.000067,-0.298536,0.283428,0.076149,0.012552,-0.032235,-0.154015,0.190935,0.027959,-0.211805,...,0.061343,-0.070200,-0.062588,-0.167686,-0.166392,-0.132634,-0.021747,-0.127389,-0.124061,-0.121054
profile_149,0.010954,-0.317524,0.245125,0.094105,0.033678,-0.012834,-0.113707,0.209848,0.063437,-0.220773,...,-0.103258,-0.303683,-0.316968,-0.201323,-0.189763,-0.183548,-0.295692,-0.162834,-0.150052,-0.167245
profile_331,0.002240,-0.236238,0.264934,0.017331,0.014026,-0.094560,-0.127930,0.073870,0.028977,-0.233251,...,-0.110462,-0.043411,-0.020181,0.252222,0.218176,0.293556,-0.031277,0.241571,0.209050,0.275704


### setup cross validation

In [8]:
alphas = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
l1_ratios = [0.1, 0.12, 0.14, 0.16, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]
n_folds = 5

In [9]:
regression_parameters = {"regress__alpha": alphas, "regress__l1_ratio": l1_ratios}

clf_parameters = {
    "classify__loss": ["log"],
    "classify__penalty": ["elasticnet"],
    "classify__alpha": alphas,
    "classify__l1_ratio": l1_ratios,
}

In [10]:
estimator_regressor = Pipeline(
    steps=[("regress", ElasticNet(random_state=42, max_iter=2000, tol=1e-3))]
)

estimator_classifier = Pipeline(
    steps=[
        (
            "classify",
            SGDClassifier(
                random_state=42,
                class_weight="balanced",
                max_iter=2000,
                shuffle=True,
                tol=1e-3,
            ),
        )
    ]
)

In [11]:
# y labels and transform instructions
cell_health_targets = y_train_df.columns.tolist()
y_transforms = ["raw", "binarize"]
binarize_fit = "sd"

### Train models 

In [12]:
%%time

cv_results_list = []
roc_results_list = []
pr_results_list = []
all_coefs_list = []
all_y_label_list = []
regression_results_list = []
cannot_fit_list = []

for cell_health_target in cell_health_targets:

    for y_transform in y_transforms:

        if y_transform == "binarize":
            estimator = estimator_classifier
            scoring = "roc_auc"
            parameters = clf_parameters
            decision_function = True
            model_type = "Classification"
        else:
            estimator = estimator_regressor
            scoring = "r2"
            parameters = regression_parameters
            decision_function = False
            model_type = "Regression"

        for shuffle_type in [True, False]:

            print("Now Training Target: {}".format(cell_health_target))
            print("[Class] Model: {}; Shuffle: {}\n".format(model_type, shuffle_type))

            # Initialize class
            chp = CellHealthPredict(
                x_df=x_train_df,
                y_df=y_train_df,
                parameters=parameters,
                estimator=estimator,
                n_folds=n_folds,
                cv_scoring=scoring,
                shuffle=shuffle_type,
            )

            # Fit model
            is_fit = chp.fit_cell_health_target(
                cell_health_target, y_transform=y_transform, binarize_fit=binarize_fit
            )

            # Identify binary threshold [default is 1.5 standard deviations]
            if y_transform == "binarize":
                binarize_threshold = chp.binarize_cutoff
            else:
                binarize_threshold = None

            if not is_fit:
                cannot_fit_list.append([cell_health_target, y_transform, shuffle_type])
                continue

            # Training performance metrics
            metric_a, metric_b, y_true, y_pred = chp.get_performance(
                decision_function=decision_function,
                return_y=True,
                binarize_fit=binarize_fit,
                binarize_fit_override=binarize_threshold,
            )

            # Testing performance metrics
            (
                metric_test_a,
                metric_test_b,
                y_test_true,
                y_test_pred,
            ) = chp.get_performance(
                x_test=x_test_df,
                y_test=y_test_df,
                decision_function=decision_function,
                return_y=True,
                binarize_fit=binarize_fit,
                binarize_fit_override=binarize_threshold,
                data_fit_type="test",
            )

            # Get Cell Line Specific Performance
            cell_line_metrics_a = []
            cell_line_metrics_b = []
            for cell_line in cell_lines:
                meta_train_subset_df = x_meta_train_df.query(
                    "Metadata_cell_line == @cell_line"
                )
                meta_test_subset_df = x_meta_test_df.query(
                    "Metadata_cell_line == @cell_line"
                )

                # Get Cell Line Specific Training Performance
                x_cell_line_df = x_train_df.reindex(
                    meta_train_subset_df.index, axis="rows"
                )
                y_cell_line_df = y_train_df.reindex(
                    meta_train_subset_df.index, axis="rows"
                )

                (
                    metric_cell_train_a,
                    metric_cell_train_b,
                    y_cell_train_true,
                    y_cell_train_pred,
                ) = chp.get_performance(
                    x_test=x_cell_line_df,
                    y_test=y_cell_line_df,
                    decision_function=decision_function,
                    return_y=True,
                    binarize_fit=binarize_fit,
                    cell_line=cell_line,
                    binarize_fit_override=binarize_threshold,
                )

                # Get Cell Line Specific Test Performance
                x_cell_line_df = x_test_df.reindex(
                    meta_test_subset_df.index, axis="rows"
                )
                y_cell_line_df = y_test_df.reindex(
                    meta_test_subset_df.index, axis="rows"
                )

                (
                    metric_cell_test_a,
                    metric_cell_test_b,
                    y_cell_test_true,
                    y_cell_test_pred,
                ) = chp.get_performance(
                    x_test=x_cell_line_df,
                    y_test=y_cell_line_df,
                    decision_function=decision_function,
                    return_y=True,
                    binarize_fit=binarize_fit,
                    cell_line=cell_line,
                    binarize_fit_override=binarize_threshold,
                    data_fit_type="test",
                )

                cell_line_metrics_a += [metric_cell_train_a, metric_cell_test_a]
                cell_line_metrics_b += [metric_cell_train_b, metric_cell_test_b]

            # Combine training and testing results
            if y_transform == "binarize":
                roc_results_list.append(
                    pd.concat([metric_a, metric_test_a], axis="rows")
                )
                roc_results_list.append(pd.concat(cell_line_metrics_a, axis="rows"))
                pr_results_list.append(
                    pd.concat([metric_b, metric_test_b], axis="rows")
                )
                pr_results_list.append(pd.concat(cell_line_metrics_b, axis="rows"))
            else:
                regression_results_list.append(
                    pd.concat([metric_a, metric_test_a], axis="rows")
                )
                regression_results_list.append(
                    pd.concat([metric_b, metric_test_b], axis="rows")
                )
                regression_results_list.append(
                    pd.concat(cell_line_metrics_a, axis="rows")
                )
                regression_results_list.append(
                    pd.concat(cell_line_metrics_b, axis="rows")
                )

            # Save cross validation results
            cv_results_list.append(chp.get_cv_results())

            # Save the model coefficients
            model_file = (
                "cell_health_{}_target_{}_shuffle_{}_transform_{}.joblib".format(
                    consensus, cell_health_target, shuffle_type, y_transform
                )
            )
            model_file = os.path.join("models", "{}_agg".format(method), model_file)
            #             model_file = os.path.join("test2","models", "{}_agg".format(method), model_file) #HYT test again 5/12
            coef_df = chp.get_coefficients(save_model=True, model_file=model_file)
            all_coefs_list.append(coef_df)

            # Store y predictions recoded values
            all_y_label_list.append(
                pd.concat([y_true, y_test_true, y_pred, y_test_pred])
            )

Now Training Target: cc_all_high_h2ax
[Class] Model: Regression; Shuffle: True



/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy s

Now Training Target: cc_all_high_h2ax
[Class] Model: Regression; Shuffle: False



/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy s

Now Training Target: cc_all_high_h2ax
[Class] Model: Classification; Shuffle: True



...

Now Training Target: vb_ros_mean
[Class] Model: Classification; Shuffle: True



/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` o

Now Training Target: vb_ros_mean
[Class] Model: Classification; Shuffle: False



/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/home/ubuntu/miniconda3/envs/cell-health-HYT/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` o

CPU times: user 4h 46min 14s, sys: 4h 37min 26s, total: 9h 23min 40s
Wall time: 36min 31s


In [13]:
# Save the models that cannot be fit
output_folder = "results/"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(f"{output_folder}/{method}_agg", exist_ok=True)

# These are the models that cannot be fit
not_fit_df = pd.DataFrame(cannot_fit_list, columns=["target", "y_transform", "shuffle"])
not_fit_file = os.path.join(
    output_folder, "{}_agg".format(method), "not_fit_models_{}.tsv".format(consensus)
)

not_fit_df.to_csv(not_fit_file, sep="\t", index=False)
not_fit_df

,target,y_transform,shuffle
0,cc_all_nucleus_roundness_mean,binarize,True
1,cc_all_nucleus_roundness_mean,binarize,False
2,cc_cc_late_mitosis,binarize,True
3,cc_cc_late_mitosis,binarize,False
4,cc_cc_s,binarize,True
5,cc_cc_s,binarize,False
6,vb_live_cell_roundness,binarize,True
7,vb_live_cell_roundness,binarize,False
8,vb_live_cell_width_length,binarize,True
9,vb_live_cell_width_length,binarize,False


In [14]:
# Acquire output metrics
full_cv_df = pd.concat(cv_results_list).reset_index(drop=True)
full_regression_results_df = pd.concat(regression_results_list).reset_index(drop=True)
full_roc_df = pd.concat(roc_results_list).reset_index(drop=True)
full_pr_df = pd.concat(pr_results_list).reset_index(drop=True)
full_coef_df = pd.concat(all_coefs_list).reset_index(drop=True)
full_y_df = pd.concat(all_y_label_list).reset_index(drop=True)

In [15]:
# Save all results
results_dir = "results/{}_agg/".format(method)
os.makedirs(results_dir, exist_ok=True)

file = os.path.join(
    results_dir, "full_cell_health_cv_results_{}.tsv.gz".format(consensus)
)
full_cv_df.to_csv(file, sep="\t", index=False)

file = os.path.join(
    results_dir, "full_cell_health_regression_{}.tsv.gz".format(consensus)
)
full_regression_results_df.to_csv(file, sep="\t", index=False)

file = os.path.join(
    results_dir, "full_cell_health_roc_results_{}.tsv.gz".format(consensus)
)
full_roc_df.to_csv(file, sep="\t", index=False)

file = os.path.join(
    results_dir, "full_cell_health_pr_results_{}.tsv.gz".format(consensus)
)
full_pr_df.to_csv(file, sep="\t", index=False)

file = os.path.join(
    results_dir, "full_cell_health_coefficients_{}.tsv.gz".format(consensus)
)
full_coef_df.to_csv(file, sep="\t", index=False)

file = os.path.join(
    results_dir, "full_cell_health_y_labels_{}.tsv.gz".format(consensus)
)
full_y_df.to_csv(file, sep="\t", index=False)